In [1]:
from pha_tools.io import gather_data_filenames, load_donation_data_from_filenames

In [2]:
data_dir = '../data/'
glob_text = 'transactions*.xlsx'
filenames = gather_data_filenames(data_dir, glob_text)
donations = load_donation_data_from_filenames(filenames)

In [3]:
years = sorted(int(yr) for yr in donations['date'].dt.year.unique())

In [4]:
def normalize_name(name):
    return ' '.join(s.capitalize() for s in name.split())

In [5]:
donations['donor_name'] = donations['donor_name'].apply(normalize_name)

In [6]:
donors_by_year = {}
for year in years:
    donors_by_year[year] = set(donations['donor_name'].loc[donations['date'].dt.year == year])

In [7]:
report = {}
for last_year in years:
    this_year = last_year + 1
    if this_year not in years: break
    this_year_donations = donations.loc[donations['date'].dt.year == this_year]
    last_year_donations = donations.loc[donations['date'].dt.year == last_year]
    
    report[this_year] = {}
    last_year_donors = donors_by_year[last_year]
    this_year_donors = donors_by_year[this_year]
    new = this_year_donors - last_year_donors
    report[this_year]['new'] = new

    lost = last_year_donors - this_year_donors
    report[this_year]['lost'] = lost

    returning = this_year_donors & last_year_donors
    report[this_year]['returning'] = returning
    report[this_year]['upgrades'] = {}  # name: (this_year_sum, last_year_sum)
    report[this_year]['downgrades'] = {}  # name: (this_year_sum, last_year_sum)
    for name in returning:
        this_year_sum = this_year_donations.loc[this_year_donations['donor_name'] == name]['amount'].sum()
        last_year_sum = last_year_donations.loc[last_year_donations['donor_name'] == name]['amount'].sum()
        year_to_year = (this_year_sum, last_year_sum)
        if this_year_sum > last_year_sum:  # upgrade if increased
            report[this_year]['upgrades'][name] = year_to_year
        else:  # no increase is a downgrade
            report[this_year]['downgrades'][name] = year_to_year

for year in report:
    print(year)
    print(f'lost: {len(report[year]["lost"])}')
    print(f'new: {len(report[year]["new"])}')
    print(f'returning: {len(report[year]["returning"])}')
    print(f'\tupgrades:   {len(report[year]['upgrades']):>4} donors for ${sum(y2y[0] - y2y[1] for name, y2y in report[year]['upgrades'].items()):9.2f}')
    print(f'\tdowngrades: {len(report[year]['downgrades']):>4} donors for ${sum(y2y[0] - y2y[1] for name, y2y in report[year]['downgrades'].items()):9.2f}')

2023
lost: 9
new: 19
returning: 301
	upgrades:    182 donors for $ 22533.53
	downgrades:  119 donors for $-11807.13
2024
lost: 10
new: 11
returning: 310
	upgrades:    185 donors for $ 22526.59
	downgrades:  125 donors for $-13247.90
2025
lost: 38
new: 8
returning: 283
	upgrades:     74 donors for $  6191.21
	downgrades:  209 donors for $-31272.77
